In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from constants import FBREF_HOMEPAGE_URL, FBREF_MATCHES_URL_PATH, PREM_LEAGUE_SCORES_URL
from fbref_scraper_match_report import load_match_soup, retrieve_match_soup_info_to_df
from fbref_all_matches_link_scraper import load_scores_table_soup, get_matches_info

In [2]:
DRIVER = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [3]:
soup = load_scores_table_soup(PREM_LEAGUE_SCORES_URL, DRIVER, tag_id = "sched_2022-2023_9_1")
df_links = get_matches_info(soup)

In [4]:
for match_uid in df_links["Match Report_uid"]:
    int_url = urljoin(FBREF_HOMEPAGE_URL, FBREF_MATCHES_URL_PATH)
    match_url = urljoin(int_url, match_uid)
    match_soup = load_match_soup(match_url, DRIVER)
    outfield_df_whole, keeper_df_whole = retrieve_match_soup_info_to_df(match_soup)
    index = 0
    for df in [outfield_df_whole, keeper_df_whole]:
        if index == 0:
            df.to_csv(f'{match_uid}_outfield.csv', index=False)
            index += 1
        else:
            df.to_csv(f'{match_uid}_keeper.csv', index=False)
    break

DRIVER.quit()

https://fbref.com/en/matches/e62f6e78
